# Model Entailment 1

This is the first of several notebooks to explore logical entailments in SysML v2 models, as well as the means by which to encode them.

In [ ]:
import json
import pymbe.api as pm

from pathlib import Path

from typing import Any, Collection, Dict, List, Tuple, Union

from pymbe.model import Model, Element
from pymbe.model_modification import create_new_classifier, own_new_element, create_new_relationship

from uuid import uuid4

## Load up basic model

Load up a basic model in order to have basic package and namespace into which to add additional elements.

In [ ]:
filename = "Model_Loader_Test_Level_1"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()).parent.parent / "tests/fixtures" / filename

level1 = pm.Model.load_from_post_file(json_file)
level1

In [ ]:
len(level1.elements)

In [ ]:
classifiers = [ele for ele in level1.elements.values() if ele._metatype == 'Classifier']
classifiers

In [ ]:
owning_membership_data_example = classifiers[0].owningRelationship._data
owning_membership_data_example

In [ ]:
base_package = classifiers[0].owningRelationship.owningRelatedElement
base_package

In [ ]:
create_new_classifier(owner=base_package, name="Trial Element", model=level1)

In [ ]:
classifiers = [ele for ele in level1.elements.values() if ele._metatype == 'Classifier']
classifiers

In [ ]:
len(level1.elements)

In [ ]:
base_package.ownedRelationship

In [ ]:
base_package.ownedMember

In [ ]:
special = [ele for ele in level1.elements.values() if ele._metatype == 'Subclassification']

In [ ]:
special[0]._data

In [ ]:
def create_super_classifier(classes : List[Element], super_name: str, owner: Element, model: Model):
    '''
    Take in a list of classifiers and generate a larger set from them. Idea is to have this work with
    individuals and classifiers of multiplicity of 1.
    '''
    new_super = create_new_classifier(owner=owner, name=super_name, model=model)
    
    for clz in classes:
        
        subclass_added_data = {
             'specific': {'@id': clz._id},
             'general': {'@id': new_super._id},
             'subclassifier': {'@id': clz._id},
             'superclassifier': {'@id': new_super._id}
        }
        
        new_sc = create_new_relationship(
            source=clz,
            target=new_super,
            owner=clz,
            model=model,
            metatype='Subclassification',
            owned_related_element=None,
            owning_related_element=clz,
            added_fields=subclass_added_data
        )
        
    return new_super
        

In [ ]:
bw1 = create_new_classifier(owner=base_package, name="Bike Wheel #1", model=level1)
bw2 = create_new_classifier(owner=base_package, name="Bike Wheel #2", model=level1)
bw3 = create_new_classifier(owner=base_package, name="Bike Wheel #3", model=level1)
bw4 = create_new_classifier(owner=base_package, name="Bike Wheel #4", model=level1)

In [ ]:
base_package.ownedMember[-1].declaredName

In [ ]:
new_wheel = create_super_classifier(classes=[bw1, bw2, bw3, bw4], super_name="Bike Wheel", model=level1, owner=base_package)

In [ ]:
new_wheel.declaredName

In [ ]:
new_wheel._data

In [ ]:
new_wheel.reverseSubclassification

In [ ]:
base_package.ownedMember

## Developing unrolling rule:
### Find Features with multiplicity of 1, typed by non-1 multiplicity types.

Look for multiplicity ranges in the model that are set to 1.

In [ ]:
def isMultiplicityOne(multiplicity_range):
    literal_value = [li.value for li in multiplicity_range.ownedElement if li['@type'] == 'LiteralInteger']
    if len(literal_value) == 0:
        return False
    elif len(literal_value) == 1:
        return literal_value[0] == 1
    elif len(literal_value) == 2:
        return literal_value[0] == 1 and literal_value[1] == 1

In [ ]:
def isMultiplicitySpecificFinite(multiplicity_range):
    literal_value = [li.value for li in multiplicity_range.ownedElement if li['@type'] == 'LiteralInteger']
    if len(literal_value) == 0:
        return False
    elif len(literal_value) == 1:
        return literal_value[0] > 1
    elif len(literal_value) == 2:
        return literal_value[0] > 1 and literal_value[0] == literal_value[1]

In [ ]:
def isTypeUndefinedMult(type_ele):
    if hasattr(element, "ownedElement") is False:
        return False
    mult_range = [mr for mr in element.ownedElement if mr['@type'] == 'MultiplicityRange']
    return len(mult_range) == 0

In [ ]:
one_multiplicities = [ele for (ele_id, ele)
                      in bicycle_model.all_non_relationships.items()
                      if ele['@type'] == 'MultiplicityRange' and isMultiplicityOne(ele)]
print (f"Found {len(one_multiplicities)} single value multiplicities")

In [ ]:
finite_multiplicities = [ele for (ele_id, ele)
                      in bicycle_model.all_non_relationships.items()
                      if ele['@type'] == 'MultiplicityRange' and isMultiplicitySpecificFinite(ele)]
print (f"Found {len(finite_multiplicities)} finite value multiplicities")

Create a function to discover multiplicities on elements.

In [ ]:
def navigateToMultiplicity(element):
    if printing_level == "TRACE":
        print(f"Trying to find multiplicity on type {element}")
    if hasattr(element, 'ownedElement'):
        mult_range = [mr for mr in element.ownedElement if mr['@type'] == 'MultiplicityRange']
        if len(mult_range) > 0:
            literal_value = [li.value for li in mult_range[0].ownedElement if li['@type'] == 'LiteralInteger']
            print(f"Found literal value on type {element}")
            return literal_value
        print(f"Found non-literal multiplicity range on type {element}")
        return mult_range
    if printing_level == "TRACE":
        print(f"Found no multiplicity on type {element}")
    return []

In [ ]:
all_class_defs = [ele for (ele_id, ele)
                   in bicycle_model.all_non_relationships.items()
                   if ele['@type'] == 'Classifier']
all_class_defs

In [ ]:
test_class = all_class_defs[2]

In [ ]:
[(each_class, navigateToMultiplicity(each_class)) for each_class in all_class_defs]

In [ ]:
all_feature_defs = [ele for (ele_id, ele)
                   in bicycle_model.all_non_relationships.items()
                   if ele['@type'] == 'Feature']

In [ ]:
model_package = "'Example Bicycle Open Wheels'"

In [ ]:
model_features = [(each_feature, 
  navigateToMultiplicity(each_feature)) for each_feature in all_feature_defs
 if each_feature.qualifiedName.split('::')[0] == model_package]
model_features

The following are functions to support the rules described in canonical instantiation.

In [ ]:
def getFiniteMultiplicityTypes(elements_list, package):
    model_types = [each_type for each_type in elements_list
                      if each_type.qualifiedName.split('::')[0] == package
                     ]
    if printing_level == "TRACE":
        print(model_types)
    return [finite_type for finite_type in model_types if
            len(navigateToMultiplicity(finite_type)) == 1 and navigateToMultiplicity(finite_type)[0] > 1]

In [ ]:
getFiniteMultiplicityTypes(all_feature_defs, model_package)

In [ ]:
getFiniteMultiplicityTypes(all_class_defs, model_package)

In [ ]:
def getSingleFeatMultiClass(features_list, package):
    if printing_level == "TRACE":
        print("Running single feature, multi class check")
    model_types = [each_type for each_type in features_list
                      if each_type.qualifiedName.split('::')[0] == package
                     ]
    single_features = [finite_type for finite_type in model_types if
            len(navigateToMultiplicity(finite_type)) == 1 and navigateToMultiplicity(finite_type)[0] == 1]
    
    if printing_level == "TRACE":
        print("Found features of single multiplicity:")
        print(single_features)
    
    multi_class = [single_feat for single_feat in single_features
                   if len(navigateToMultiplicity(single_feat.type[0])) == 0 or 
                   len(getFiniteMultiplicityTypes(single_feat.type, package)) > 0]
    
    if printing_level == "TRACE":
        print("Found features of single multiplicity with non-single multiplicity types:")
        print(multi_class)
    
    return multi_class

In [ ]:
single_features_list = getSingleFeatMultiClass(all_feature_defs, model_package)

In [ ]:
single_features_list

## Defining Unrolling Rule:
### Subset the non-1 multiplicity type and then redefine features to have the new 1-multiplicity types type them

In [ ]:
# TODO: Need to develop methods for adding model elements in PyMBE library.